In [ ]:
import os
import csv
import random

# Path to the dataset
dataset_path = r"C:\Users\PMLS\Documents\2ndSemester\CV\cvProject\PSL\UAlpha"

# Mapping English folder names to Urdu labels
english_to_urdu = {
    "Alif": "ا",
    "Bay": "ب",
    "Pay": "پ",
    "Tay": "ت",
    "Taay": "ٹ",
    "Say": "ث",
    "Chay": "چ",
    "1-Hay": "ح",
    "Khay": "خ",
    "Dal": "د",
    "Daal": "ڈ",
    "Zaal": "ذ",
    "Ray": "ر",
    "Zay": "ز",
    "Zaey": "ژ",
    "Seen": "س",
    "Sheen": "ش",
    "Suad": "ص",
    "Zuad": "ض",
    "Tuey": "ط",
    "Zuey": "ظ",
    "Ain": "ع",
    "Ghain": "غ",
    "Fay": "ف",
    "Kaf": "ق",
    "Kiaf": "ک",
    "Gaaf": "گ",
    "Lam": "ل",
    "Meem": "م",
    "Nuun": "ن",
    "Nuungh": "ں",
    "Wao": "و",
    "Dochahay": "ھ",
    "Hamza": "ء",
    "Cyeh": "ى",
    "Byeh": "ے"
}

# Output CSV files
train_csv = "train_dataset.csv"
test_csv = "test_dataset.csv"

# Train-test split ratio
train_ratio = 0.8

# Prepare CSV data
data = []

for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    if os.path.isdir(folder_path) and folder_name in english_to_urdu:
        urdu_label = english_to_urdu[folder_name]
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            data.append([image_path, urdu_label])

# Shuffle the data
random.shuffle(data)

# Split the data
train_size = int(len(data) * train_ratio)
train_data = data[:train_size]
test_data = data[train_size:]

# Write to train CSV
with open(train_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["image_path", "label"])  # Header
    writer.writerows(train_data)

# Write to test CSV
with open(test_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["image_path", "label"])  # Header
    writer.writerows(test_data)

print(f"Train CSV file created: {train_csv}")
print(f"Test CSV file created: {test_csv}")


In [ ]:
import tensorflow as tf
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
from tensorflow import keras

# Paths to CSV files and dataset folder
train_csv = "train_dataset.csv"
test_csv = "test_dataset.csv"
dataset_path = r"C:\Users\PMLS\Documents\2ndSemester\CV\cvProject\PSL\UAlpha"

# Image processing parameters
image_size = (224, 224)
batch_size = 32
epochs = 10

# Function to preprocess image
def preprocess_image(image_path, label):
    # Load and preprocess the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.keras.applications.mobilenet_v3.preprocess_input(image)
    return image, label

# Function to load and preprocess data from CSV
def load_dataset(csv_file, label_encoder):
    df = pd.read_csv(csv_file)
    # Ensure the image_path column contains valid paths (no need to prepend dataset_path)
    
    # Encode labels
    df['label'] = label_encoder.transform(df['label'])
    
    # Create a TensorFlow dataset
    image_paths = df['image_path'].values
    labels = df['label'].values
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load the training and test datasets
label_encoder = LabelEncoder()

# Encode labels using all data (train + test for consistency)
all_labels = pd.concat([pd.read_csv(train_csv)['label'], pd.read_csv(test_csv)['label']])
label_encoder.fit(all_labels)

train_dataset = load_dataset(train_csv, label_encoder)
test_dataset = load_dataset(test_csv, label_encoder)

# Define MobileNetV3 model
def create_mobilenet_v3_model(num_classes):
    base_model = tf.keras.applications.MobileNetV3Large(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze the base model

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Create and train the model
num_classes = len(label_encoder.classes_)
model = create_mobilenet_v3_model(num_classes)

# Train the model
model.fit(train_dataset, epochs=epochs)

# Save the trained model
model.save("mobilenet_v3_urdu_alphabets.h5")
keras.saving.save_model(model, 'my_model.keras')

# Evaluate the model
test_images, test_labels = [], []
for img, lbl in test_dataset.unbatch():
    test_images.append(img.numpy())
    test_labels.append(lbl.numpy())
test_images = np.array(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)

# Generate classification report
class_names = label_encoder.classes_
report = classification_report(test_labels, y_pred, target_names=class_names)
print("Classification Report:\n", report)


In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow import keras

# Paths to CSV files
train_csv = "train_dataset.csv"
test_csv = "test_dataset.csv"

# Image processing parameters
image_size = (224, 224)
batch_size = 32
epochs = 2

# Preprocess image function
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.keras.applications.efficientnet_v2.preprocess_input(image)
    return image, label

# Load dataset
def load_dataset(csv_file, label_encoder):
    df = pd.read_csv(csv_file)
    df['label'] = label_encoder.transform(df['label'])
    image_paths = df['image_path'].values
    labels = df['label'].values
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Prepare datasets
label_encoder = LabelEncoder()
all_labels = pd.concat([pd.read_csv(train_csv)['label'], pd.read_csv(test_csv)['label']])
label_encoder.fit(all_labels)

train_dataset = load_dataset(train_csv, label_encoder)
test_dataset = load_dataset(test_csv, label_encoder)

# Create model
def create_efficientnet_model(num_classes):
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax', dtype="float32")(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Model creation and training
num_classes = len(label_encoder.classes_)
model = create_efficientnet_model(num_classes)
model.fit(train_dataset, epochs=epochs)

# Fine-tuning
base_model = model.get_layer(index=0)
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(train_dataset, epochs=5)

# Save model
model.save("efficientnet_urdu_alphabets.keras")
model.save("efficientb0_urdu_alphabets.h5")
# Evaluation
results = model.evaluate(test_dataset, batch_size=batch_size)
print(f"Test Accuracy: {results[1] * 100:.2f}%")

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model = tf.keras.models.load_model("efficientnet_urdu_alphabets.keras", compile=False)

# Define label names for predictions
labels = [
    "ء", "ا", "ب", "ت", "ث", "ح", "خ", "د", "ذ", "ر", "ز", "س", "ش", "ص", "ض",
    "ط", "ظ", "ع", "غ", "ف", "ق", "ل", "م", "ن", "و", "ى", "ٹ", "پ", "چ", "ڈ",
    "ژ", "ک", "گ", "ں", "ھ", "ے"
]

# Camera settings
camera = cv2.VideoCapture(0)
box_size = 224  # Square box dimensions for capturing hand region
x1, y1 = 100, 100  # Top-left corner of the box

print("Press 'c' to capture an image for prediction.")
print("Press 'q' to quit the application.")

while True:
    ret, frame = camera.read()
    if not ret:
        break

    # Flip the frame for a mirror effect
    frame = cv2.flip(frame, 1)

    # Draw a bounding box on the frame
    x2, y2 = x1 + box_size, y1 + box_size
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display instructions on the screen
    cv2.putText(frame, "Press 'c' to capture, 'q' to quit", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Show the frame
    cv2.imshow("Sign Language Detection", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):  # Capture image for prediction
        roi = frame[y1:y2, x1:x2]

        # Preprocess the ROI for prediction
        img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = tf.keras.applications.efficientnet.preprocess_input(img.astype(np.float32))
        img = np.expand_dims(img, axis=0)

        # Predict the sign
        predictions = model.predict(img)
        predicted_label = labels[np.argmax(predictions)]

        # Display the prediction on the frame
        cv2.putText(frame, f"Predicted: {predicted_label}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Show the prediction
        print(f"Predicted: {predicted_label}")

    elif key == ord('q'):  # Quit the application
        break

# Release resources
camera.release()
cv2.destroyAllWindows()

In [ ]:
import tensorflow as tf
import pandas as pd
import os
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Paths to CSV files and dataset folder
train_csv = "train_dataset.csv"
test_csv = "test_dataset.csv"
dataset_path = r"C:\Users\PMLS\Documents\2ndSemester\CV\cvProject\PSL\UAlpha"

# Image processing parameters
image_size = (224, 224)
batch_size = 32
epochs = 10

# Function to preprocess image
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.keras.applications.mobilenet_v3.preprocess_input(image)
    return image, label

# Function to load and preprocess data from CSV
def load_dataset(csv_file, label_encoder):
    df = pd.read_csv(csv_file)
    df['label'] = label_encoder.transform(df['label'])
    image_paths = df['image_path'].values
    labels = df['label'].values
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load the training and test datasets
label_encoder = LabelEncoder()
all_labels = pd.concat([pd.read_csv(train_csv)['label'], pd.read_csv(test_csv)['label']])
label_encoder.fit(all_labels)

train_dataset = load_dataset(train_csv, label_encoder)
test_dataset = load_dataset(test_csv, label_encoder)

# Define MobileNetV3 model
def create_mobilenet_v3_model(num_classes):
    base_model = tf.keras.applications.MobileNetV3Large(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Create and train the model
num_classes = len(label_encoder.classes_)
model = create_mobilenet_v3_model(num_classes)

model.fit(train_dataset, epochs=epochs)

# Save the model in Keras format
model.save("mobilenet_v3_urdu_alphabets.h5")
#tf.keras.saving.save_model(model, 'mobilenet_v3_model.keras')


In [ ]:

# Save the model in Pickle format
import pickle
with open("mobilenet_v3_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Evaluate the model
test_images, test_labels = [], []
for img, lbl in test_dataset.unbatch():
    test_images.append(img.numpy())
    test_labels.append(lbl.numpy())
test_images = np.array(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)

# Generate and save the classification report
class_names = label_encoder.classes_
report = classification_report(test_labels, y_pred, target_names=class_names, output_dict=True)
with open("CLASSFICATIONreportmbilenetv3.json", "w") as f:
    json.dump(report, f, indent=4)
print("Classification Report saved to CLASSFICATIONreportmbilenetv3.json")

# Generate and display confusion matrix
cm = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
# Save confusion matrix as an image
plt.savefig("confusion_matrix_mobilenet_v3.png")

In [3]:
from tensorflow.keras.models import load_model

# Define the file path to save the model
keras_model_save_path = 'saved_mobilenetv3_model.keras'

# Save the model in Keras format (.h5)
model.save(keras_model_save_path)
print(f"Model saved at {keras_model_save_path}")


Model saved at saved_mobilenetv3_model.keras


In [ ]:
import tensorflow as tf
import pandas as pd
import os
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Paths to CSV files and dataset folder
train_csv = "train_dataset.csv"
test_csv = "test_dataset.csv"
dataset_path = r"C:\Users\PMLS\Documents\2ndSemester\CV\cvProject\PSL\UAlpha"

# Image processing parameters
image_size = (224, 224)
batch_size = 64
epochs = 10

# Function to preprocess image
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

# Function to load and preprocess data from CSV
def load_dataset(csv_file, label_encoder):
    df = pd.read_csv(csv_file)
    df['label'] = label_encoder.transform(df['label'])
    image_paths = df['image_path'].values
    labels = df['label'].values
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load the training and test datasets
label_encoder = LabelEncoder()
all_labels = pd.concat([pd.read_csv(train_csv)['label'], pd.read_csv(test_csv)['label']])
label_encoder.fit(all_labels)

train_dataset = load_dataset(train_csv, label_encoder)
test_dataset = load_dataset(test_csv, label_encoder)

# Define MobileNetV2 model
def create_mobilenet_v2_model(num_classes):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Create and train the model
num_classes = len(label_encoder.classes_)
model = create_mobilenet_v2_model(num_classes)

model.fit(train_dataset, epochs=epochs)

# Save the model in Keras format
model.save("mobilenet_v2_urdu_alphabets.h5")
print("Model saved in Keras format as mobilenet_v2_urdu_alphabets.h5")

# Save the model in Pickle format
import pickle
with open("mobilenet_v2_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("Model saved in Pickle format as mobilenet_v2_model.pkl")

# Evaluate the model
test_images, test_labels = [], []
for img, lbl in test_dataset.unbatch():
    test_images.append(img.numpy())
    test_labels.append(lbl.numpy())
test_images = np.array(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)

# Generate and save the classification report
class_names = label_encoder.classes_
report = classification_report(test_labels, y_pred, target_names=class_names, output_dict=True)
with open("CLASSFICATIONreportmobilenetv2.json", "w") as f:
    json.dump(report, f, indent=4)
print("Classification Report saved to CLASSFICATIONreportmobilenetv2.json")

# Generate and display confusion matrix
cm = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Save confusion matrix as an image
plt.savefig("confusion_matrix_mobilenet_v2.png")
print("Confusion Matrix saved as confusion_matrix_mobilenet_v2.png")

In [ ]:

# Save the model in Keras format
model.save("mobilenet_v2_urdu_alphabets.h5")
print("Model saved in Keras format as mobilenet_v2_urdu_alphabets.h5")

# Save the model in Pickle format
import pickle
with open("mobilenet_v2_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("Model saved in Pickle format as mobilenet_v2_model.pkl")

# Evaluate the model
test_images, test_labels = [], []
for img, lbl in test_dataset.unbatch():
    test_images.append(img.numpy())
    test_labels.append(lbl.numpy())
test_images = np.array(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)

# Generate and save the classification report
class_names = label_encoder.classes_
report = classification_report(test_labels, y_pred, target_names=class_names, output_dict=True)
# Generate classification report
class_names = label_encoder.classes_
report = classification_report(test_labels, y_pred, target_names=class_names)
print("Classification Report:\n", report)
with open("CLASSFICATIONreportmobilenetv2.json", "w") as f:
    json.dump(report, f, indent=4)
print("Classification Report saved to CLASSFICATIONreportmobilenetv2.json")

# Generate and display confusion matrix
cm = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Save confusion matrix as an image
plt.savefig("confusion_matrix_mobilenet_v2.png")
print("Confusion Matrix saved as confusion_matrix_mobilenet_v2.png")


In [ ]:
# Generate classification report
class_names = label_encoder.classes_
report = classification_report(test_labels, y_pred, target_names=class_names)
print("Classification Report:\n", report)

In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Paths to CSV files and dataset folder
train_csv = "train_dataset.csv"
test_csv = "test_dataset.csv"
dataset_path = r"C:\Users\PMLS\Documents\2ndSemester\CV\cvProject\PSL\UAlpha"

# Image processing parameters
image_size = (224, 224)
batch_size = 32
epochs = 10

# Function to preprocess image
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image, label

# Function to load and preprocess data from CSV
def load_dataset(csv_file, label_encoder):
    df = pd.read_csv(csv_file)
    df['label'] = label_encoder.transform(df['label'])
    image_paths = df['image_path'].values
    labels = df['label'].values
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load the training and test datasets
label_encoder = LabelEncoder()
all_labels = pd.concat([pd.read_csv(train_csv)['label'], pd.read_csv(test_csv)['label']])
label_encoder.fit(all_labels)

train_dataset = load_dataset(train_csv, label_encoder)
test_dataset = load_dataset(test_csv, label_encoder)

# Define EfficientNet model
def create_efficientnet_model(num_classes):
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze base model weights

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Create and train the model
num_classes = len(label_encoder.classes_)
model = create_efficientnet_model(num_classes)

# Training with ModelCheckpoint to save the best model
checkpoint_path = "efficientnet_model_best.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# Train the model
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[checkpoint]
)

# Save the model in Keras format
model.save("efficientnet_model.h5")

# Save the model in Pickle format
import pickle
with open("efficientnet_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Evaluate the model
test_images, test_labels = [], []
for img, lbl in test_dataset.unbatch():
    test_images.append(img.numpy())
    test_labels.append(lbl.numpy())
test_images = np.array(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)

# Generate and save the classification report
class_names = label_encoder.classes_
report = classification_report(test_labels, y_pred, target_names=class_names, output_dict=True)
print("Classification Report:\n", report)
with open("classification_report_efficientnet.json", "w") as f:
    json.dump(report, f, indent=4)
print("Classification Report saved to classification_report_efficientnet.json")

# Generate and display confusion matrix
cm = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig("confusion_matrix_efficientnet.png")
plt.show()


In [ ]:

# Save the model in Keras format
model.save("efficientnet_model.h5")

# Save the model in Pickle format
import pickle
with open("efficientnet_modelNEW.pkl", "wb") as f:
    pickle.dump(model, f)

# Evaluate the model
test_images, test_labels = [], []
for img, lbl in test_dataset.unbatch():
    test_images.append(img.numpy())
    test_labels.append(lbl.numpy())
test_images = np.array(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)

# Generate and save the classification report
class_names = label_encoder.classes_
report = classification_report(test_labels, y_pred, target_names=class_names, output_dict=True)
print("Classification Report:\n", report)

with open("classification_report_efficientnet.json", "w") as f:
    json.dump(report, f, indent=4)
print("Classification Report saved to classification_report_efficientnet.json")

# Generate and display confusion matrix
cm = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig("confusion_matrix_efficientnet.png")
plt.show()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from collections import Counter
import pickle

# === Load Models ===
# Load model saved as a .pkl file
model_path = "mobilenet_v2_model.pkl"
with open(model_path, "rb") as f:
    model2 = pickle.load(f)

# Load other models
model1 = tf.keras.models.load_model("efficientnet_urdu_alphabets.keras", compile=False)
model3 = tf.keras.models.load_model("saved_mobilenetv3_model.keras", compile=False)

# === Label Definitions ===
labels = [
    "ء", "ا", "ب", "ت", "ث", "ح", "خ", "د", "ذ", "ر", "ز", "س", "ش", "ص", "ض",
    "ط", "ظ", "ع", "غ", "ف", "ق", "ل", "م", "ن", "و", "ى", "ٹ", "پ", "چ", "ڈ",
    "ژ", "ک", "گ", "ں", "ھ", "ے"
]

# === Camera Settings ===
camera = cv2.VideoCapture(0)
if not camera.isOpened():
    print("Error: Unable to access the camera.")
    exit()

box_size = 224
x1, y1 = 100, 100  # Top-left corner of the bounding box

print("Press 'c' to capture an image for prediction.")
print("Press 'q' to quit the application.")

# === Main Loop ===
while True:
    ret, frame = camera.read()
    if not ret:
        print("Error: Unable to read from the camera.")
        break

    # Flip frame for a mirror effect
    frame = cv2.flip(frame, 1)

    # Define and draw bounding box
    x2, y2 = x1 + box_size, y1 + box_size
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display instructions
    cv2.putText(frame, "Press 'c' to capture, 'q' to quit", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Show the live video feed
    cv2.imshow("Sign Language Detection", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):  # Capture image
        roi = frame[y1:y2, x1:x2]

        # Preprocess the ROI
        img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = tf.keras.applications.efficientnet.preprocess_input(img.astype(np.float32))
        img = np.expand_dims(img, axis=0)

        # Get predictions from all models
        predictions1 = model1.predict(img)
        predictions2 = model2.predict(img)
        predictions3 = model3.predict(img)

        # Get the predicted labels
        pred_label1 = labels[np.argmax(predictions1)]
        pred_label2 = labels[np.argmax(predictions2)]
        pred_label3 = labels[np.argmax(predictions3)]

        # Perform ensemble prediction using majority voting
        final_prediction = Counter([pred_label1, pred_label2, pred_label3]).most_common(1)[0][0]

        # Overlay predictions on the frame
        cv2.putText(frame, f"Model 1: {pred_label1}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        cv2.putText(frame, f"Model 2: {pred_label2}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        cv2.putText(frame, f"Model 3: {pred_label3}", (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        cv2.putText(frame, f"Final Prediction: {final_prediction}", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display predictions in the console
        print(f"Model 1 Prediction: {pred_label1}")
        print(f"Model 2 Prediction: {pred_label2}")
        print(f"Model 3 Prediction: {pred_label3}")
        print(f"Ensemble Prediction: {final_prediction}")

        # Display the updated frame
        cv2.imshow("Sign Language Detection", frame)

    elif key == ord('q'):  # Quit
        break

# === Cleanup ===
camera.release()
cv2.destroyAllWindows()
